In [1]:
# -*- coding: utf-8 -*-
# !/usr/bin/env python
# coding: utf-8

# Importing python packages
import numpy as np
import pandas as pd

In [2]:
# Load content from Google drive
from google.colab import drive
drive.mount("/content/drive")
path_to_data = '/content/drive/MyDrive/Predictive_models_AV_AM/Modeling/'      # For Arijit
# path_to_data = '/content/drive/MyDrive/ColabNotebooks/BacteriaSelection/'     # For Arun

Mounted at /content/drive


In [3]:
# Read CSV file with single and pairs bacteria data
data_one = pd.read_csv(path_to_data + 'one.csv')
data_two = pd.read_csv(path_to_data + 'two.csv')

# Print unioue values in the column
bacterias = data_one['Strains'].unique()
print("Bacterias:", bacterias)

# Changing data with unique one-hot encoding
c_size = 5
cluster_map = {'P31D':0, '9B2':0,
        '9B1':1, '6A1':1, '3C2':1, '9X2':1,
        '6A2':2, '8X4':2, '10B2':2, '4C':2, '9F3':2,
        'P32B1':3, 'P33G':3, '1A1':3, '7F21':3, '8A2':3,
        '8X1':4, '3C1':4}

bact_map = {0: ['P31D', '9B2'],
        1: ['9B1', '6A1', '3C2', '9X2'],
        2: ['6A2', '8X4', '10B2', '4C', '9F3'],
        3: ['P32B1', 'P33G', '1A1', '7F21', '8A2'],
        4: ['8X1', '3C1']}

# value_map = {'P31D':0.6, '9B2':0.4,
#         '9B1':0.8, '6A1':0.65, '3C2':0.5, '9X2':0.35,
#         '6A2':0.85, '8X4':0.7, '10B2':0.55, '4C':0.4, '9F3':0.25,
#         'P32B1':0.85, 'P33G':0.7, '1A1':0.55, '7F21':0.4, '8A2':0.25,
#         '8X1':0.55, '3C1':0.45}

# cluster_map = {'P31D':0, '9B2':0,
#         '9B1':1, '6A1':1, '3C2':1, '9X2':1, '6A2':1, '8X4':1, '10B2':1, '4C':1, '9F3':1,
#         'P32B1':2, 'P33G':2, '1A1':2, '7F21':2, '8A2':2, '8X1':2, '3C1':2}

# value_map = {'P31D':0.6, '9B2':0.4,
#         '9B1':0.85, '6A1':0.75, '3C2':0.65, '9X2':0.55, '6A2':0.5, '8X4':0.45, '10B2':0.4, '4C':0.35, '9F3':0.3,
#         'P32B1':0.85, 'P33G':0.75, '1A1':0.65, '7F21':0.55, '8A2':0.45, '8X1':0.35, '3C1':0.25}

# data_one['Strains'] = data_one['Strains'].map(dict)
# data_two['Strain1'] = data_two['Strain1'].map(dict)
# data_two['Strain2'] = data_two['Strain2'].map(dict)

Bacterias: ['8A2' 'P33G' '9X2' '9B2' '1A1' '4C' '8X4' 'P31D' '8X1' '10B2' '7F21'
 '6A2' '3C1' '9F3' '9B1' '3C2' 'P32B1' '6A1']


In [4]:
# Read CSV file and convert 18 dimentional datapoints
data_np = data_one.to_numpy()

# Print the first row of the data
avg_responses_one = {}
count             = 0
sum               = 0
for n in range(len(data_np)):
    r = data_np[n]
    sum += r[1]
    count += 1
    if n < len(data_np) - 1:
        if data_np[n+1][2] == 1:
          avg_responses_one[r[0]] = sum/count
          sum = 0
          count = 0

    else:
        # print (count)
        avg_responses_one[r[0]] = sum/count

value_map = {}
for c in bact_map:
    bact = bact_map[c]
    max_response = -1
    for b in bact:
        if max_response < avg_responses_one[b]:
            max_response = avg_responses_one[b]

    for b in bact:
        value_map[b] = avg_responses_one[b]/max_response

# Print number of samples
# print (avg_responses_one)
# print (dict(sorted(avg_responses_one.items(), key=lambda item: item[1])))
print (value_map)

{'P31D': 0.5824484735091375, '9B2': 1.0, '9B1': 0.6796272496638376, '6A1': 0.9767929715778826, '3C2': 1.0, '9X2': 0.41672537282315236, '6A2': 0.7903111733786583, '8X4': 1.0, '10B2': 0.6759648102582029, '4C': 0.480689774504968, '9F3': 0.5717217709369325, 'P32B1': 0.895047466203349, 'P33G': 1.0, '1A1': 0.5660737043126777, '7F21': -0.033916689304862135, '8A2': 0.2109719437984934, '8X1': 1.0, '3C1': 0.32727273033520654}


In [5]:
# Read CSV file and convert 18 dimentional datapoints
data_np = data_one.to_numpy()

# Print the first row of the data
queries           = np.array(np.zeros(c_size))
responses         = np.array([0])
avg_queries_one   = np.array(np.zeros(c_size))
avg_responses_one = np.array([0])
count             = 0
sum               = 0
for n in range(len(data_np)):
    r = data_np[n]
    x = np.zeros(c_size)
    x[cluster_map[r[0]]] = value_map[r[0]]
    queries     = np.vstack((queries, x))
    responses   = np.vstack((responses, np.array([r[1]])))
    sum += r[1]
    count += 1
    if n < len(data_np) - 1:
        if data_np[n+1][2] == 1:
          # print (count)
          avg_queries_one = np.vstack((avg_queries_one, x))
          avg_responses_one = np.vstack((avg_responses_one, np.array([sum/count])))
          sum = 0
          count = 0

    else:
        # print (count)
        avg_queries_one = np.vstack((avg_queries_one, x))
        avg_responses_one = np.vstack((avg_responses_one, np.array([sum/count])))

        # Removing dummy data
        queries = np.delete(queries, 0, axis=0)
        responses = np.delete(responses, 0, axis=0)
        avg_queries_one = np.delete(avg_queries_one, 0, axis=0)
        avg_responses_one = np.delete(avg_responses_one, 0, axis=0)

# Print number of samples
print (len(queries), len(avg_queries_one))

101 18


In [6]:
# Read CSV file with two bacterias
data_two_np = data_two.to_numpy()

# Print the first row of the data
queries_two       = np.array(np.zeros(c_size))
responses_two     = np.array([0])
avg_queries_two   = np.array(np.zeros(c_size))
avg_responses_two = np.array([0])
count             = 0
sum               = 0
for n in range(len(data_two_np)):
    r = data_two_np[n]
    x = np.zeros(c_size)
    x[cluster_map[r[0]]] = value_map[r[0]]
    x[cluster_map[r[1]]] += value_map[r[1]]
    queries_two     = np.vstack((queries_two, x))
    responses_two   = np.vstack((responses_two, np.array([r[2]])))
    sum += r[2]
    count += 1
    if n < len(data_two_np) - 1:
        if data_two_np[n+1][3] == 1:
            # print (count)
            avg_queries_two = np.vstack((avg_queries_two, x))
            avg_responses_two = np.vstack((avg_responses_two, np.array([sum/count])))
            sum = 0
            count = 0

    else:
        # print (count)
        avg_queries_two = np.vstack((avg_queries_two, x))
        avg_responses_two = np.vstack((avg_responses_two, np.array([sum/count])))

        # Removing dummy data
        queries_two = np.delete(queries_two, 0, axis=0)
        responses_two = np.delete(responses_two, 0, axis=0)
        avg_queries_two = np.delete(avg_queries_two, 0, axis=0)
        avg_responses_two = np.delete(avg_responses_two, 0, axis=0)

# Print number of samples
print (len(queries_two), len(avg_queries_two))

185 35


In [7]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.7 MB/s eta 0:00:00


In [18]:
### Different ML models ###
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, Matern
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor

import catboost as cb

import lightgbm as lgb


# Linear Regression
linear_reg = LinearRegression()

# Support Vector Regression
svm_reg = SVR(kernel="linear", C=1, epsilon=0.1)

# Ridge Regression
linear_ridge = Ridge(alpha=0.1)

# Ridge regression with polynomial kernel with degree = 2
poly_reg = make_pipeline(PolynomialFeatures(2), Ridge(alpha=0.1))

# Decision Tree Regressor
tree_reg = DecisionTreeRegressor(max_depth=3)

# Random Forest Regression
forest_reg = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=100)

# Gradient Boosting Regression
grad_reg = GradientBoostingRegressor(n_estimators=500, max_depth=10)

# Bagging Regression
bag_reg = BaggingRegressor(estimator=RandomForestRegressor(n_estimators=10, max_depth=5), n_estimators=20)

# XGBoost Regression
xgb_reg = XGBRegressor(n_estimators=200, max_depth=3)

# K-nearest neighbors Regression model
knn_reg = KNeighborsRegressor(n_neighbors=5)

# Gaussian process with Matérn kernel as surrogate model
m52 = ConstantKernel(1.0) * Matern(length_scale=1.0, nu=2.5)
gpr_mr = GaussianProcessRegressor(kernel=m52, alpha=0.1**2, optimizer='fmin_l_bfgs_b')

# Gaussian process with RBF kernel as surrogate model
kernel = C(1.0, (1e-3, 1e3)) * RBF(10, (1e-2, 1e2))
gpr_rbf = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=20, optimizer='fmin_l_bfgs_b')

# Neural Network
mlp_reg = MLPRegressor(hidden_layer_sizes=(100, 100), max_iter=1000)

# CatBoost Regression
cat_reg = cb.CatBoostRegressor(iterations=100, learning_rate=0.1, depth=5, verbose=0)

# LightGBM Regressor
lgb_reg = lgb.LGBMRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, verbose=-1)

# Model to try
model_info = [
    ["Linear Regression\t", linear_reg],
    ["Support Vector Regression", svm_reg],
    ["Ridge Regression\t", linear_ridge],
    ["Ridge with Poly kernel (d=2)", poly_reg],
    ["Decision Tree Regressor\t", tree_reg],
    ["Random Forest Regression", forest_reg],
    ["Gradient Boosting Regression", grad_reg],
    ["Bagging Regression\t", bag_reg],
    ["XGBoost Regression\t", xgb_reg],
    ["K-nearest neighbors Regression", knn_reg],
    ["Multi-layer Neural Network", mlp_reg],
    ["GP with RBF kernel\t", gpr_rbf],
    ["GP with Matérn kernel\t", gpr_mr],
    ["CatBoost Regression\t", cat_reg],
    ["LightGBM Regression\t", lgb_reg]
]


In [19]:
# ### Supress warning ###
import warnings
from sklearn.exceptions import ConvergenceWarning

# Suppress only the ConvergenceWarning
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)

# ### Average data with test accuracy on unseen data ###
# Splitting data
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(
    avg_queries_two, avg_responses_two, test_size=0.33, random_state=100)

Y_train = Y_train.ravel()
Y_test = Y_test.ravel()

# Difference model performance
print ("Model \t\t\t\t Train R-squared \t Train RMSE \t\t Train MAE \t\t Test R-squared \t Test RMSE \t\t Test MAE \n")
for m in model_info:
  model_name = m[0]
  model = m[1]
  if model_name != "LightGBM Regression\t":
    model.fit(X_train, Y_train)

  else:
    model.fit(X_train, Y_train, categorical_feature=[0, 1, 2, 3, 4])

  # Model predictions
  y_pred_train = model.predict(X_train)
  y_pred = model.predict(X_test)

  # Root mean square mean error
  rmse_train = np.sqrt(np.mean((Y_train - y_pred_train)**2))
  rmse = np.sqrt(np.mean((Y_test - y_pred)**2))

  # R-squared value
  r2_train = r2_score(Y_train, y_pred_train)
  r2 = r2_score(Y_test, y_pred)

  # Mean Absolute error
  mae_train = mean_absolute_error(Y_train, y_pred_train)
  mae = mean_absolute_error(Y_test, y_pred)

  # Print
  print("{}\t{}\t{}\t{}\t{}\t{}\t{}".format(model_name, r2_train, rmse_train, mae_train, r2, rmse, mae))

Model 				 Train R-squared 	 Train RMSE 		 Train MAE 		 Test R-squared 	 Test RMSE 		 Test MAE 

Linear Regression		0.15677227889696876	0.17558043081508512	0.13227697529432825	-1.2829837967735296	0.3541896732683914	0.2899063762412105
Support Vector Regression	0.11419878810752604	0.1799582692146459	0.1366273691155715	-0.5134315288958524	0.28838048562518603	0.24236111899162982
Ridge Regression		0.15321337143032854	0.1759505660381057	0.1322018314764735	-0.666502260004447	0.30261289734198404	0.2496477685750381
Ridge with Poly kernel (d=2)	0.5144064510729449	0.1332416882460103	0.09354093030701044	-1.0101301340023263	0.3323506766281626	0.23073981953052314
Decision Tree Regressor		0.578945434414459	0.1240717371689733	0.08855879823707695	0.37851882106812695	0.18479851230050004	0.1460519909707004
Random Forest Regression	0.7770737407210052	0.09027849007991068	0.07636692608054291	0.4428442442206675	0.1749737062004119	0.12345655494729786
Gradient Boosting Regression	0.9242432072854369	0.052627708

## **Using model to get estimated growth for untested combinations**

-- Writing **Machine learning models** part on page 31
-- Why does Tree model work better, citation
-- Generate data for all pairs

In [20]:
# ### Predicting the outcome for combination of m bacterias ###
combinations = np.array(np.zeros(c_size))
bact_names = []
for b in range(len(bacterias)):
  for n in range(b+1, len(bacterias)):
    x = np.zeros(c_size)
    x[cluster_map[bacterias[b]]] = value_map[bacterias[b]]
    x[cluster_map[bacterias[n]]] += value_map[bacterias[n]]
    combinations = np.vstack((combinations, x))
    bact_names.append([bacterias[b], bacterias[n]])

# Removing dummy data
combinations = np.delete(combinations, 0, axis=0)

# Predictations of the best model
best_model = forest_reg
y_pred_poly = best_model.predict(combinations).flatten()

print(len(combinations))

153


In [21]:
# Sort by predicted plant growth (high to low)
sorted_ind = np.argsort(y_pred_poly)[::-1]
y_pred_sorted = y_pred_poly[sorted_ind]
bact_names_sorted = np.array(bact_names)[sorted_ind]

print("{:<5} {:<25} {:>5}".format("Rank", "Bacteria combination", "Predicted plant growth"), end="\n")

for c in range(len(combinations)):
    bact = str(tuple(bact_names_sorted[c]))
    # Create a new row as a dictionary
    new_row = {"Rank":[c+1], "Bacteria combination":[bact], "Predicted plant growth":[y_pred_sorted[c]]}

    # Append the new row to the DataFrame
    print("{:<5} {:<25} {:>5}".format(c+1, bact, y_pred_sorted[c]), end="\n")


Rank  Bacteria combination      Predicted plant growth
1     ('10B2', '6A2')           0.5920440337460473
2     ('1A1', '9B1')            0.5686766480469725
3     ('9X2', '1A1')            0.5683033234757557
4     ('9B1', '6A1')            0.5538942357622445
5     ('3C2', '6A1')            0.5538942357622445
6     ('9B1', '3C2')            0.5538942357622445
7     ('9X2', '3C2')            0.5538942357622445
8     ('9X2', '6A1')            0.5538942357622445
9     ('1A1', '3C2')            0.5449191718441733
10    ('1A1', '6A1')            0.5443369932441733
11    ('9X2', '9B1')            0.5396796366540079
12    ('1A1', '7F21')           0.5269105608569524
13    ('1A1', 'P31D')           0.5224850219028435
14    ('9B2', '1A1')            0.5164651683362179
15    ('1A1', '8X1')            0.5101950219634341
16    ('1A1', '3C1')            0.5101950219634341
17    ('8A2', '9B1')            0.5090576767285571
18    ('P33G', '9B1')           0.49428728173559966
19    ('9B1', 'P32B1')    

## **Finding best hyper-parameters**

In [10]:
print ("Decision Tree Regressor\t\t Train R-squared \t Train RMSE \t\t Train MAE \t\t Test R-squared \t Test RMSE \t\t Test MAE \n")

# Hyper-paramter search for Decision Tree Regressor
max_depth_list = [1, 2, 3, 4, 5]

for d in max_depth_list:
  model = DecisionTreeRegressor(max_depth=d)
  model.fit(X_train, Y_train)

  # Model predictions
  y_pred_train = model.predict(X_train)
  y_pred = model.predict(X_test)

  # Root mean square mean error
  rmse_train = np.sqrt(np.mean((Y_train - y_pred_train)**2))
  rmse = np.sqrt(np.mean((Y_test - y_pred)**2))

  # R-squared value
  r2_train = r2_score(Y_train, y_pred_train)
  r2 = r2_score(Y_test, y_pred)

  # Mean Absolute error
  mae_train = mean_absolute_error(Y_train, y_pred_train)
  mae = mean_absolute_error(Y_test, y_pred)

  # Print
  print("(d={})\t\t\t\t{}\t{}\t{}\t{}\t{}\t{}".format(d, r2_train, rmse_train, mae_train, r2, rmse, mae))


Decision Tree Regressor		 Train R-squared 	 Train RMSE 		 Train MAE 		 Test R-squared 	 Test RMSE 		 Test MAE 

(d=1)				0.2672847306464351	0.1636708333087865	0.12370361856507224	0.24316390488840978	0.2039320093411444	0.1672085594079629
(d=2)				0.48075351814601974	0.13778135576012002	0.10821588983448552	0.3541150357961248	0.18839183116844382	0.15267714271117014
(d=3)				0.5789454344144589	0.12407173716897331	0.08855879823707694	0.37851882106812695	0.18479851230050004	0.1460519909707004
(d=4)				0.7001378945162503	0.10470428744044606	0.0752873317022771	0.24021761382159434	0.20432856775083835	0.155091111586625
(d=5)				0.8806928662956679	0.06604453375438359	0.03886485683899275	0.24737471003954847	0.20336391031255402	0.13335562847318058


In [17]:
print ("Decision Tree Regressor\t\t Train R-squared \t Train RMSE \t\t Train MAE \t\t Test R-squared \t Test RMSE \t\t Test MAE \n")

# Hyper-paramter search for Decision Tree Regressor
max_depth_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
n_estimators_list = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]

for e in n_estimators_list:
  for d in max_depth_list:
    model = RandomForestRegressor(n_estimators=e, max_depth=d, random_state=100)

    model.fit(X_train, Y_train)

    # Model predictions
    y_pred_train = model.predict(X_train)
    y_pred = model.predict(X_test)

    # Root mean square mean error
    rmse_train = np.sqrt(np.mean((Y_train - y_pred_train)**2))
    rmse = np.sqrt(np.mean((Y_test - y_pred)**2))

    # R-squared value
    r2_train = r2_score(Y_train, y_pred_train)
    r2 = r2_score(Y_test, y_pred)

    # Mean Absolute error
    mae_train = mean_absolute_error(Y_train, y_pred_train)
    mae = mean_absolute_error(Y_test, y_pred)

    # Print
    print("(e={},d={})\t\t\t{}\t{}\t{}\t{}\t{}\t{}".format(e, d, r2_train, rmse_train, mae_train, r2, rmse, mae))

Decision Tree Regressor		 Train R-squared 	 Train RMSE 		 Train MAE 		 Test R-squared 	 Test RMSE 		 Test MAE 

(e=100,d=1)			0.28519413656776915	0.16165819512050117	0.12360960370256388	0.216763985890713	0.20745829509522834	0.1702134338498563
(e=100,d=2)			0.5051421356430887	0.13450669783282265	0.10688927510637461	0.3484926779634022	0.1892100195889323	0.14980491378448194
(e=100,d=3)			0.6448303581814339	0.11395190533046531	0.09534699570769478	0.40489681388779164	0.18083422028356352	0.13636151443282826
(e=100,d=4)			0.7298236876009342	0.09938647955999351	0.08515895601519712	0.430477652726608	0.17690490112303875	0.12661066904014648
(e=100,d=5)			0.7770737407210052	0.09027849007991068	0.07636692608054291	0.4428442442206675	0.1749737062004119	0.12345655494729786
(e=100,d=6)			0.7968803147741986	0.0861746801148029	0.07205884871657915	0.4386076098767351	0.17563769981437893	0.12259917385173819
(e=100,d=7)			0.8012076790952147	0.08525178374357366	0.07111485569146785	0.4395772782810511	0.175485